In [ ]:
import os
os.environ['OMP_NUM_THREADS'] = 6

In [4]:
# copied from https://huggingface.co/bigcode/starcoder2-7b

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

checkpoint = "bigcode/starcoder2-7b"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# for fp16 use `torch_dtype=torch.float16` instead
model = AutoModelForCausalLM.from_pretrained(checkpoint, torch_dtype=torch.bfloat16, low_cpu_mem_usage=False)

inputs = tokenizer.encode("def print_hello_world():", return_tensors="pt")#.to("cuda")
outputs = model.generate(inputs)
print(tokenizer.decode(outputs[0]))

model.safetensors.index.json:   0%|          | 0.00/41.6k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.89G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.51G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:49152 for open-end generation.
/home/matthew/.virtualenvs/llm-experiments/lib/python3.11/site-packages/transformers/generation/utils.py:1181: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


def print_hello_world():
    print("Hello World!")

def print_hello_


In [8]:
inputs.shape

torch.Size([1, 7])

In [9]:
outputs.shape

torch.Size([1, 20])

In [10]:
model

Starcoder2ForCausalLM(
  (model): Starcoder2Model(
    (embed_tokens): Embedding(49152, 4608)
    (layers): ModuleList(
      (0-31): 32 x Starcoder2DecoderLayer(
        (self_attn): Starcoder2SdpaAttention(
          (q_proj): Linear(in_features=4608, out_features=4608, bias=True)
          (k_proj): Linear(in_features=4608, out_features=512, bias=True)
          (v_proj): Linear(in_features=4608, out_features=512, bias=True)
          (o_proj): Linear(in_features=4608, out_features=4608, bias=True)
          (rotary_emb): Starcoder2RotaryEmbedding()
        )
        (mlp): Starcoder2MLP(
          (c_fc): Linear(in_features=4608, out_features=18432, bias=True)
          (c_proj): Linear(in_features=18432, out_features=4608, bias=True)
          (act): PytorchGELUTanh()
        )
        (input_layernorm): LayerNorm((4608,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((4608,), eps=1e-05, elementwise_affine=True)
      )
    )
    (norm): LayerNo

In [11]:
gg = model.generate(outputs, max_length=100)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:49152 for open-end generation.


In [12]:
gg

tensor([[  610,  1489,   100,  7670,   100,  5879,  2284,   303,  1489,   459,
          8302, 10914, 16013,   222,   222,   610,  1489,   100,  7670,   100,
          5879,   100,  1814,   100,   444,    45,   444,   731,   303,  1489,
           459,  8302,   332,   494,   655,   494,   332, 16013,   222,   222,
           610,  1489,   100,  7670,   100,  5879,   100,  1814,   100,   444,
           100,   382,   100,   400,    45,   444,    49, 11505,   731,   303,
          1489,   459,  8302,   332,   494,   655,   494,   332, 16013,   303,
          1489,   459,  4296,   904,   332,   494,   615,    45,   400,    46,
           494,   332, 11339,  3627,  7164,   222,   222,   610,  1489,   100,
          7670,   100,  5879,   100,  1814,   100,   444,   100,   382,   100]])

In [14]:
print(tokenizer.decode(gg[0]))

def print_hello_world():
    print("Hello World!")

def print_hello_world_with_name(name):
    print("Hello " + name + "!")

def print_hello_world_with_name_and_age(name, age):
    print("Hello " + name + "!")
    print("You are " + str(age) + " years old.")

def print_hello_world_with_name_and_


In [18]:
bad = '''
def broken():
    print(a)
    a = 123

def fixed():
'''

In [19]:
b2 = model.generate(tokenizer.encode(bad, return_tensors='pt'), max_length=50)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:49152 for open-end generation.


In [20]:
print(tokenizer.decode(b2[0]))


def broken():
    print(a)
    a = 123

def fixed():
	global a
	a = 123

broken()
fixed()
<file_sep>/python/python_basics/0


In [21]:
b3 = model.generate(tokenizer.encode(bad, return_tensors='pt'), max_length=50)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:49152 for open-end generation.


In [24]:
help(model.generate)

Help on method generate in module transformers.generation.utils:

generate(inputs: Optional[torch.Tensor] = None, generation_config: Optional[transformers.generation.configuration_utils.GenerationConfig] = None, logits_processor: Optional[transformers.generation.logits_process.LogitsProcessorList] = None, stopping_criteria: Optional[transformers.generation.stopping_criteria.StoppingCriteriaList] = None, prefix_allowed_tokens_fn: Optional[Callable[[int, torch.Tensor], List[int]]] = None, synced_gpus: Optional[bool] = None, assistant_model: Optional[ForwardRef('PreTrainedModel')] = None, streamer: Optional[ForwardRef('BaseStreamer')] = None, negative_prompt_ids: Optional[torch.Tensor] = None, negative_prompt_attention_mask: Optional[torch.Tensor] = None, **kwargs) -> Union[transformers.generation.utils.GenerateDecoderOnlyOutput, transformers.generation.utils.GenerateEncoderDecoderOutput, transformers.generation.utils.GenerateBeamDecoderOnlyOutput, transformers.generation.utils.GenerateBe

In [23]:
print(tokenizer.decode(b3[0]))


def broken():
    print(a)
    a = 123

def fixed():
	global a
	a = 123

broken()
fixed()
<file_sep>/python/python_basics/0


In [25]:
fib = '''
(defn fibonacci [n]
'''
f1 = model.generate(tokenizer.encode(fib, return_tensors='pt'), max_length=50)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:49152 for open-end generation.


In [27]:
print(tokenizer.decode(f1[0]))


(defn fibonacci [n]
(if (<= n 1)
n
(+ (fibonacci (- n 1)) (fibonacci (- n 2)))))

(fibonacci 10)
55



In [ ]:
f2 = model.generate(